In [10]:
import numpy as np
import pandas as pd
from regex import search

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

import warnings
warnings.filterwarnings("ignore")

### Reading in artist data

In [11]:
# Reading in full dataset (includes wikipedia and spotify data), one row per group member
df = pd.read_csv('../../extraction/wiki_and_spoti_merged.csv')
df['clean_genre'] =list(map(lambda x: x[14:-6] if (str(x)[0:5] =='/wiki') else x, df.genre))
print(df.shape)
df.head()

(28680, 35)


,Unnamed: 0,member,attr,band,sections,years active,breakups,origin,member years active,genre,...,mean_song_danceability,mean_song_acousticness,mean_song_energy,mean_song_instrumentalness,mean_song_liveness,mean_song_loudness,mean_song_speechiness,mean_song_tempo,unwanted_member,clean_genre
0,0,Joey Eppard,current,3 (American band),8,['1994-present'],0.0,"Woodstock, New York, U.S.",NaN,_progressive_metal_bands,...,0.854,0.641,0.3738,0.000017,0.174,-8.8538,0.17198,122.502,False,_progressive_metal_bands
1,1,Billy Riker,current,3 (American band),8,['1994-present'],0.0,"Woodstock, New York, U.S.",NaN,_progressive_metal_bands,...,0.854,0.641,0.3738,0.000017,0.174,-8.8538,0.17198,122.502,False,_progressive_metal_bands
2,2,Daniel Grimsland,current,3 (American band),8,['1994-present'],0.0,"Woodstock, New York, U.S.",NaN,_progressive_metal_bands,...,0.854,0.641,0.3738,0.000017,0.174,-8.8538,0.17198,122.502,False,_progressive_metal_bands
3,3,Chris Gartmann,current,3 (American band),8,['1994-present'],0.0,"Woodstock, New York, U.S.",NaN,_progressive_metal_bands,...,0.854,0.641,0.3738,0.000017,0.174,-8.8538,0.17198,122.502,False,_progressive_metal_bands
4,4,Joe Cuchelo,past,3 (American band),8,['1994-present'],0.0,"Woodstock, New York, U.S.",NaN,_progressive_metal_bands,...,0.854,0.641,0.3738,0.000017,0.174,-8.8538,0.17198,122.502,False,_progressive_metal_bands


In [12]:
# Frequency of genres
df.clean_genre.value_counts()

_metalcore_bands                          3629
_Christian_metal_bands                    3563
_power_metal_bands                        2725
_glam_metal_bands_and_artists             2198
_nu_metal_bands                           1857
_alternative_metal_bands                  1692
_industrial_metal_bands                   1661
_symphonic_metal_bands                    1616
_doom_metal_bands                         1576
_progressive_metal_bands                  1469
_gothic_metal_bands                       1070
_groove_metal_bands                        876
_melodic_death_metal_bands                 840
_folk_metal_bands                          729
_new_wave_of_British_heavy_metal_bands     700
_grindcore_bands                           686
_deathcore_bands                           663
_Viking_metal_bands                        550
_speed_metal_bands                         379
_mathcore_bands                            201
Name: clean_genre, dtype: int64

In [13]:
# List of genres
genres = df.clean_genre.value_counts().index

### Reading in group data

In [16]:
# One row only per group
group_by_group_features = pd.read_csv('../../extraction/final_spotify_df.csv')
group_by_group_features.head()

,Unnamed: 0,Group_name_(wiki),Group_name_(spoti),Spotify_id,Group_popularity,Group_followers,count_albums,mean_tracks_per_album,total_tracks,earliest_release_date,...,min_song_popularity,max_song_popularity,mean_song_danceability,mean_song_acousticness,mean_song_energy,mean_song_instrumentalness,mean_song_liveness,mean_song_loudness,mean_song_speechiness,mean_song_tempo
0,0,Powerman 5000,Powerman 5000,spotify:artist:5imUS9dQyCbAjUEJJ9QyWC,56.0,503221.0,20.0,3.85,77.0,2019-10-19,...,45.0,68.0,0.5652,0.000966,0.9304,5.853200e-03,0.16228,-3.6016,0.08618,139.7978
1,1,Norma Jean,Norma Jean,spotify:artist:26YwXMnl5wfwgt4km8cFud,21.0,894.0,20.0,3.85,77.0,2019-10-19,...,13.0,23.0,0.6118,0.598200,0.3906,2.280000e-07,0.25028,-12.2248,0.04154,133.0340
2,2,ASP,ASP,spotify:artist:4seqkIdZS9cuf74cVESXiL,22.0,1865.0,20.0,3.85,77.0,2019-10-19,...,10.0,14.0,0.3734,0.003466,0.9604,3.991600e-01,0.42100,-3.0336,0.06564,139.8112
3,3,Eluveitie,Eluveitie,spotify:artist:5X0N2k3qMnI8kSrGJT3kfT,58.0,443421.0,20.0,3.85,77.0,2019-10-19,...,52.0,60.0,0.4300,0.015037,0.9408,8.733920e-03,0.49864,-4.3454,0.08362,117.3440
4,4,Holy Blood,Holy Blood,spotify:artist:1tTt8DFLyX4sUKIvYnMg0x,8.0,5822.0,20.0,3.85,77.0,2019-10-19,...,4.0,9.0,0.3170,0.004852,0.9472,6.806000e-01,0.23854,-5.1952,0.06648,156.0076


In [17]:
for current_genre in genres:

    ###### Creating subset only for a specific genre
    df_genre = df[df['clean_genre']==current_genre]

    ###### Extract only the group names that correspond to a genre
    group_names = list(df_genre['Group_name_(wiki)'].value_counts().index)

    ###### Subset groups within a given genre
    subset_groups_agg_df = df_genre[df_genre['Group_name_(wiki)'].isin(group_names)]
    subset_groups_agg_df['genre'] = current_genre

    ###### Give a number to each group
    subset_groups_agg_df = subset_groups_agg_df.reset_index()
    subset_groups_agg_df = subset_groups_agg_df.drop(['index','Unnamed: 0'], axis=1)
    subset_groups_agg_df['band_number'] = subset_groups_agg_df.index + 1

    ###### Assign band number to each artist
    bands_number_mapping = subset_groups_agg_df[['Group_name_(wiki)', 'band_number']]
    df_genre = df_genre.merge(bands_number_mapping, how='left', on='Group_name_(wiki)')
    df_genre_subset = df_genre[['Group_name_(wiki)', 'attr' , 'origin']].drop_duplicates()
    df_genre_subset = df_genre_subset.groupby('Group_name_(wiki)').head(1)

    ###### Clean origin
    df_genre_subset['origin_clean'] = list(map(lambda x: str(x).split(',')[len(str(x).split(','))-1], 
                                   df_genre_subset['origin']))
    df_genre_subset['origin_clean'] = np.where(df_genre_subset['origin_clean']==' United States', 
                                         ' U.S.', df_genre_subset['origin_clean'])

    ###### Merge back origin to group agg df
    subset_groups_agg_df = subset_groups_agg_df.merge(df_genre_subset, how='left', on='Group_name_(wiki)')

    ###### Produce group aggregate df only for segment
    subset_groups_agg_df = subset_groups_agg_df.fillna(0)
    csv_path_agg_df = str('SEGMENT_group_features_') + str(current_genre) + str('.csv')
    subset_groups_agg_df.to_csv(csv_path_agg_df)

    ##### Identify artists who participate to more than 1 group (serve as connections)
    # Count of different groups per artist
    moved_artists = pd.DataFrame([df_genre.member.value_counts().index, df_genre.member.value_counts().values]).T
    moved_artists = moved_artists.rename({0: 'member', 1: 'count'}, axis='columns')
    # subset of artists who have belonged to multiple groups
    mult_group_artists = moved_artists[moved_artists['count']>1]

    ###### Identify all group connections
    all_groups = []
    for one_member in mult_group_artists.member:

        # Zoom in on all groups for one member
        subset = df_genre[df_genre['member'] == one_member]

        # Find the group numbers for this artist
        groups = list(subset.band_number.value_counts().index)
        groups.sort()
        groups

        for i in range(0, len(groups)):
            for j in range(i+1, (len(groups))):
                # print(str(groups[i]) + '-' + str(groups[j]))
                all_groups.append(str(groups[i]) + '-' + str(groups[j]))

    ##### Summary of connection pairs
    group_pairs_df = pd.DataFrame(all_groups, columns = ['group_pairs'])
    pair = group_pairs_df.group_pairs.value_counts().index 
    count_connections = group_pairs_df.group_pairs.value_counts().values
    group_pairs = pd.DataFrame([pair, count_connections]).T
    group_pairs = group_pairs.rename({0: 'groups', 1: 'count_connections'}, axis='columns')

    #### Finalize pairs df
    group_pairs['groups'] = group_pairs['groups'].astype(str)
    group_pairs['group_1'] = list(map(lambda x: int(x.split('-')[0]), group_pairs['groups']))
    group_pairs['group_2'] = list(map(lambda x: int(x.split('-')[1]), group_pairs['groups']))
    group_pairs_final = group_pairs[['group_1', 'group_2']]

    #### Saving group pairs to csv
    csv_path_group_pairs = str('SEGMENT_group_pairs_') + str(current_genre) + str('.csv')
    group_pairs_final.to_csv(csv_path_group_pairs, index=False)

In [18]:
group_pairs_final

,group_1,group_2
0,171,186
1,172,183
2,183,186
3,182,185
4,171,173
...,...,...
878,23,25
879,28,29
880,27,30
881,29,30


## Demonstration

Overlap between groups

In [7]:
group_pairs_final

,group_1,group_2
0,12,14
1,11,17


In [9]:
members_band_1 = df_genre[df_genre['band_number']==group_pairs_final['group_1'][0]].member

In [10]:
members_band_2 = df_genre[df_genre['band_number']==group_pairs_final['group_2'][0]].member

In [11]:
set(members_band_1).intersection(members_band_2)

{' Michael Ehré', 'Ralf Scheepers'}

In [12]:
current_genre

'speed_metal'